In [6]:
# 本代码由可视化策略环境自动生成 2022年5月25日 15:47
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


def m14_generate_sql_bigquant_run():
    # 使用sql读取数据，数量限制在10000行内
#    sql = "SELECT a.ts_code as instrument, a.trade_date as date, a.open, a.high, a.low, a.close, a.vol as volume, a.amount, b.market_name, b.ts_code as index_code, b.w_score, b.bias_cs, b.bias_sm, b.bias_ml, b.bias_cs_dd, b.bias_sm_dd, b.bias_ml_dd, b.cs_diff1, b.sm_diff1, b.ml_diff1, b.cs_diff2, b.sm_diff2, b.ml_diff2, c.adj_factor FROM 嘉翼基金池日线数据 as a, 嘉翼1号指数基金宽度数据 as b, 嘉翼基金池复权因子 as c where b.trade_ts_code = a.ts_code and a.ts_code = c.ts_code and a.trade_date = b.trade_date and a.trade_date = c.trade_date and a.trade_date >= '2017-01-01' order by date"
    df = "SELECT * from 嘉翼1号指数基金宽度数据 union SELECT * from 嘉翼2号指数基金宽度数据"
    
    return df
    # return ""

def m3_run_bigquant_run(input_1, w_score_l, w_score_h):
    df = input_1.read_df()
    df.rename(columns = {'trade_ts_code': 'instrument', 'trade_date': 'date'}, inplace=True)
    
    # 清理 df
    df['date'] = pd.to_datetime(df['date'])
    df['instrument'] = df['instrument'].apply(lambda x: x[:7]+'HOF' if x[-2:] == 'SH' else x)
    df['instrument'] = df['instrument'].apply(lambda x: x[:7]+'ZOF' if x[-2:] == 'SZ' else x)
    df[df.select_dtypes(np.float64).columns] = df.select_dtypes(np.float64).astype(np.float32)
    del df['Unnamed: 0']
    
    instruments = set(df['instrument'].tolist())
    start_date = df['date'].min().strftime('%Y-%m-%d')
    end_date = df['date'].max().strftime('%Y-%m-%d')
    
    df_hloc = DataSource("bar1d_CN_FUND").read(instruments, start_date=start_date, end_date=end_date)   
    
    # 行情数据与因子数据合并
    df_his = df_hloc.merge(df, on=['date', 'instrument'], how='left')
    
    # 数据补全
    df_his.sort_values(by=['instrument', 'date'], inplace=True)
    df_his.fillna(method='ffill', inplace=True)
    df_his.dropna(inplace=True)
    
    # 输出准备
#     df_1 = df_his[['instrument','date','open','high','low','close', 'volume', 'amount']]
#     df_2 = df_his[['instrument','date','w_score','bias_cs','bias_sm','bias_ml','bias_cs_dd','bias_sm_dd','bias_ml_dd','cs_diff1','sm_diff1','ml_diff1','cs_diff2','sm_diff2','ml_diff2']]
    df_his.sort_values(by=['instrument', 'date'], inplace=True)
    df_his['w_score_1'] = df_his['w_score'].shift(1)
    df_his['bias_cs_1'] = df_his['bias_cs'].shift(1)
    df_his['bias_cs_dd_1'] = df_his['bias_cs_dd'].shift(1)
    df_his = df_his[['instrument','date','open','close','w_score','w_score_1','bias_cs','bias_cs_1','bias_cs_dd','bias_cs_dd_1']]
    df_his['buy_condition'] = 0
    df_his['sell_condition'] = 0
    
    # ----------------------------- 绝对宽度逻辑 ----------------------------------#
    def cal_con(x):
        # 买点逻辑，昨日收盘后，宽度 w_score < 10， 短期乖离 bias_cs < 0，短期乖离回撤 bias_cs_dd > 0.05
        # 卖点逻辑，昨日收盘后，宽度 w_score > 90， 短期乖离 bias_cs > 0，短期乖离回撤 bias_cs_dd < -0.05
        x['buy_condition'] = 1 if (x['w_score_1']<w_score_l)&(x['bias_cs_1']<0)&(x['bias_cs_dd_1']>0.05)&(x['bias_cs_1']!=x['bias_cs_dd_1']!=0) else 0
        x['sell_condition'] = -1 if (x['w_score_1']>w_score_h)&(x['bias_cs_1']>0)&(x['bias_cs_dd_1']<-0.05)&(x['bias_cs_1']!=x['bias_cs_dd_1']!=0) else 0
        return x
    df_his = df_his.apply(cal_con, axis = 1)
    
    basic_info = {'market':'CN_FUND', 'instruments':list(instruments), 'start_date':start_date, 'end_date':end_date}  # 构建bq需要的配置结构字典
    data_1 = DataSource.write_pickle(basic_info)  # 代码列表
    data_2 = DataSource.write_df(df_his)  # 行情数据
    
    return Outputs(data_1=data_1, data_2=data_2, data_3=None)

# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m3_post_run_bigquant_run(outputs):
    return outputs

# 回测引擎：初始化函数，只执行一次
def m15_initialize_bigquant_run(context):
    # 系统已经设置了默认的交易手续费和滑点，要修改手续费可使用如下函数
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    
# 回测引擎：每日数据处理函数，每天执行一次
def m15_handle_data_bigquant_run(context, data):
    # 获取今日的日期
    today = data.current_dt.strftime('%Y-%m-%d')  
    # 通过positions对象，使用列表生成式的方法获取目前持仓的股票列表
    stock_hold_now = {e.symbol: p.amount * p.last_sale_price for e, p in context.perf_tracker.position_tracker.positions.items()}
    # 记录用于买入股票的可用现金,因为是早盘卖股票，需要记录卖出的股票市值并在买入下单前更新可用现金；如果是早盘买尾盘卖，则卖出时不需更新可用现金，因为尾盘卖出股票所得现金无法使用
    cash_for_buy = context.portfolio.cash
    
    try:
        buy_stock = context.daily_stock_buy[today]  # 当日符合买入条件的股票
    except:
        buy_stock=[]  # 如果没有符合条件的股票，就设置为空
    
    try:
        sell_stock = context.daily_stock_sell[today]  # 当日符合卖出条件的股票
    except:
        sell_stock=[] # 如果没有符合条件的股票，就设置为空
    
    # 需要卖出的股票:已有持仓中符合卖出条件的股票
    stock_to_sell = [ i for i in stock_hold_now if i in sell_stock ]
    # 需要买入的股票:没有持仓且符合买入条件的股票
#     stock_to_buy = [ i for i in buy_stock if i not in stock_hold_now ]  
    stock_to_buy = [ i for i in buy_stock]
    
    # 如果有卖出信号
    if len(stock_to_sell)>0:
#         print(f"--------------------------------------------\n当前日期{today}\n持仓明细：{stock_hold_now}\n持有现金{cash_for_buy}\n当日计划卖出：{stock_to_sell}")
        for instrument in stock_to_sell:
            sid = context.symbol(instrument) # 将标的转化为equity格式
            cur_position = context.portfolio.positions[sid].amount # 持仓
            if cur_position > 0 and data.can_trade(sid):
#                 context.order_target_percent(sid, -0.25) # 卖出 
                context.order_value(sid, -1000)
                # 因为设置的是早盘卖出早盘买入，需要根据卖出的股票更新可用现金；如果设置尾盘卖出早盘买入，则不需更新可用现金(可以删除下面的语句)
                cash_for_buy += stock_hold_now[instrument]
    
    # 如果有买入信号/有持仓
    if len(stock_to_buy)>0:
#         print(f"--------------------------------------------\n当前日期{today}\n持仓明细：{stock_hold_now}\n持有现金{cash_for_buy}\n当日计划买入：{stock_to_buy}")
        for instrument in stock_to_buy:
            sid = context.symbol(instrument) # 将标的转化为equity格式
            if  cash_for_buy > 0 and data.can_trade(sid):
                context.order_value(sid, 2000)
                cash_for_buy -= 2000
# 回测引擎：准备数据，只执行一次
def m15_prepare_bigquant_run(context):
    # 加载预测数据
    df = context.options['data'].read_df()

    # 函数：求满足开仓条件的股票列表
    def open_pos_con(df):
        return list(df[df['buy_condition']==1].instrument)

    # 函数：求满足平仓条件的股票列表
    def close_pos_con(df):
        return list(df[df['sell_condition']==-1].instrument)

    # 每日买入股票的数据框
    context.daily_stock_buy = df.groupby('date').apply(open_pos_con)
  
    # 每日卖出股票的数据框
    context.daily_stock_sell = df.groupby('date').apply(close_pos_con)
# 回测引擎：每个单位时间开始前调用一次，即每日开盘前调用一次。
def m15_before_trading_start_bigquant_run(context, data):
    pass

def m1_run_bigquant_run(input_1, rolling_day=120, w_score_percent_l=0.1, w_score_percent_h=0.9):
    df = input_1.read_df()
    df.rename(columns = {'trade_ts_code': 'instrument', 'trade_date': 'date'}, inplace=True)
    
    # 清理 df
    df['date'] = pd.to_datetime(df['date'])
    df['instrument'] = df['instrument'].apply(lambda x: x[:7]+'HOF' if x[-2:] == 'SH' else x)
    df['instrument'] = df['instrument'].apply(lambda x: x[:7]+'ZOF' if x[-2:] == 'SZ' else x)
    df[df.select_dtypes(np.float64).columns] = df.select_dtypes(np.float64).astype(np.float32)
    del df['Unnamed: 0']
    
    instruments = set(df['instrument'].tolist())
    start_date = df['date'].min().strftime('%Y-%m-%d')
    end_date = df['date'].max().strftime('%Y-%m-%d')
    
    df_hloc = DataSource("bar1d_CN_FUND").read(instruments, start_date=start_date, end_date=end_date)   
    
    # 行情数据与因子数据合并
    df_his = df_hloc.merge(df, on=['date', 'instrument'], how='left')
    
    # 数据补全
    df_his.sort_values(by=['instrument', 'date'], inplace=True)
    df_his.fillna(method='ffill', inplace=True)
    df_his.dropna(inplace=True)
    
    # 输出准备
#     df_1 = df_his[['instrument','date','open','high','low','close', 'volume', 'amount']]
#     df_2 = df_his[['instrument','date','w_score','bias_cs','bias_sm','bias_ml','bias_cs_dd','bias_sm_dd','bias_ml_dd','cs_diff1','sm_diff1','ml_diff1','cs_diff2','sm_diff2','ml_diff2']]
    df_his.sort_values(by=['instrument', 'date'], inplace=True)
    df_his['w_score_1'] = df_his['w_score'].shift(1)
    df_his['bias_cs_1'] = df_his['bias_cs'].shift(1)
    df_his['bias_cs_dd_1'] = df_his['bias_cs_dd'].shift(1)
    df_his = df_his[['instrument','date','open','close','w_score','w_score_1','bias_cs','bias_cs_1','bias_cs_dd','bias_cs_dd_1']]
    df_his['buy_condition'] = 0
    df_his['sell_condition'] = 0
    
    # ----------------------------- 加权宽度逻辑 ----------------------------------#
    def cal_con(x):
        # 买点逻辑，昨日收盘后，宽度 w_score < 过去120天的10分位， 短期乖离 bias_cs < 0，短期乖离回撤 bias_cs_dd > 0.05
        # 卖点逻辑，昨日收盘后，宽度 w_score > 过去120天的90分位， 短期乖离 bias_cs > 0，短期乖离回撤 bias_cs_dd < -0.05
        x['buy_condition'] = 1 if (x['w_score_1']<x['w_score_120_l_1'])&(x['bias_cs_1']<0)&(x['bias_cs_dd_1']>0.05)&(x['bias_cs_1']!=x['bias_cs_dd_1']!=0) else 0
        x['sell_condition'] = -1 if (x['w_score_1']>x['w_score_120_h_1'])&(x['bias_cs_1']>0)&(x['bias_cs_dd_1']<-0.05)&(x['bias_cs_1']!=x['bias_cs_dd_1']!=0) else 0
        return x

    def percent(x, level=None):
        return x.quantile(q=level)

    df_all = pd.DataFrame()
    for i in range(len(list(instruments))):
        df = df_his[df_his['instrument']==list(instruments)[i]].copy()
        df['w_score_120_l'] = df['w_score'].rolling(rolling_day).apply(percent, args=(w_score_percent_l,))
        df['w_score_120_h'] = df['w_score'].rolling(rolling_day).apply(percent, args=(w_score_percent_h,))

        df['w_score_120_l_1'] = df['w_score_120_l'].shift(1)
        df['w_score_120_h_1'] = df['w_score_120_h'].shift(1)

        df.dropna(inplace=True)    
        df = df.apply(cal_con, axis=1)

        if df_all.empty:
            df_all = df
        else:
            df_all = pd.concat([df_all, df], axis=0, join='outer', ignore_index=True)
    start_date = df_all['date'].min().strftime('%Y-%m-%d')
    end_date = df_all['date'].max().strftime('%Y-%m-%d')
    
    basic_info = {'market':'CN_FUND', 'instruments':list(instruments), 'start_date':start_date, 'end_date':end_date}  # 构建bq需要的配置结构字典
    data_1 = DataSource.write_pickle(basic_info)  # 代码列表
    data_2 = DataSource.write_df(df_all)  # 行情数据
    
    return Outputs(data_1=data_1, data_2=data_2, data_3=None)

# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m1_post_run_bigquant_run(outputs):
    return outputs


g = T.Graph({

    'm14': 'M.datahub_load_postgres.v1',
    'm14.conn': 'postgres:Cong2021@DB009@121.41.121.43:5433',
    'm14.database': 'jyquant',
    'm14.generate_sql': m14_generate_sql_bigquant_run,

    'm31': 'M.df_to_csv.v2',
    'm31.input_1': T.Graph.OutputPort('m14.data'),
    'm31.name': '300ext_his.csv',

    'm32': 'M.input_csv.v5',
    'm32.file': '300ext_his.csv',
    'm32.coding': 'UTF-8',
    'm32.dtypes': {},
    'm32.date_type': '%Y-%m-%d',
    'm32.date_cols': ['trade_date'],

    'm3': 'M.cached.v3',
    'm3.input_1': T.Graph.OutputPort('m32.data'),
    'm3.run': m3_run_bigquant_run,
    'm3.post_run': m3_post_run_bigquant_run,
    'm3.input_ports': 'input_1',
    'm3.params': """{
    'w_score_l': 0.1, 
    'w_score_h': 0.9,
}""",
    'm3.output_ports': 'data_1, data_2',

    'm15': 'M.trade.v4',
    'm15.instruments': T.Graph.OutputPort('m3.data_1'),
    'm15.options_data': T.Graph.OutputPort('m3.data_2'),
    'm15.start_date': '2017-12-13',
    'm15.end_date': '2022-04-01',
    'm15.initialize': m15_initialize_bigquant_run,
    'm15.handle_data': m15_handle_data_bigquant_run,
    'm15.prepare': m15_prepare_bigquant_run,
    'm15.before_trading_start': m15_before_trading_start_bigquant_run,
    'm15.volume_limit': 0,
    'm15.order_price_field_buy': 'open',
    'm15.order_price_field_sell': 'open',
    'm15.capital_base': 100000,
    'm15.auto_cancel_non_tradable_orders': False,
    'm15.data_frequency': 'daily',
    'm15.price_type': '后复权',
    'm15.product_type': '股票',
    'm15.plot_charts': True,
    'm15.backtest_only': False,
    'm15.benchmark': '000300.HIX',

    'm1': 'M.cached.v3',
    'm1.run': m1_run_bigquant_run,
    'm1.post_run': m1_post_run_bigquant_run,
    'm1.input_ports': 'input_1',
    'm1.params': """{
    'rolling_day' : 120, 
    'w_score_percent_l' : 0.1, 
    'w_score_percent_h' : 0.9,
}""",
    'm1.output_ports': '',
})

# g.run({})


def m2_param_grid_builder_bigquant_run():
    param_grid = {}

    # 在这里设置需要调优的参数备选
    param_grid['m1.rolling_day'] = [30, 60, 120, 180, 220]
    param_grid['m1.w_score_percent_l'] = [0.01, 0.05, 0.1, 0.2]
    param_grid['m1.w_score_percent_h'] = [0.99, 0.95, 0.9, 0.8]
    
    return param_grid

def m2_scoring_bigquant_run(result):
    score = result.get('m15').read_raw_perf()['sharpe'].tail(1)[0]

    return {'score': score}


m2 = M.hyper_parameter_search.v1(
    param_grid_builder=m2_param_grid_builder_bigquant_run,
    scoring=m2_scoring_bigquant_run,
    search_algorithm='网格搜索',
    search_iterations=5,
    workers=1,
    worker_distributed_run=True,
    worker_silent=True,
    run_now=True,
    bq_graph=g
)


[2022-05-05 10:37:59.382856] INFO: moduleinvoker: input_csv.v5 开始运行..

[2022-05-05 10:37:59.394190] INFO: moduleinvoker: 命中缓存

[2022-05-05 10:37:59.395998] INFO: moduleinvoker: input_csv.v5 运行完成[0.013157s].

[2022-05-05 10:37:59.416088] INFO: moduleinvoker: cached.v3 开始运行..

[2022-05-05 10:37:59.432715] INFO: moduleinvoker: 命中缓存

...

[2022-05-05 10:38:08.190199] INFO: Performance: Simulated 1045 trading days out of 1045.

[2022-05-05 10:38:08.191668] INFO: Performance: first open: 2017-12-13 09:30:00+00:00

[2022-05-05 10:38:08.192808] INFO: Performance: last close: 2022-04-01 15:00:00+00:00

[2022-05-05 10:38:23.350149] INFO: moduleinvoker: backtest.v8 运行完成[23.859619s].

[2022-05-05 10:38:23.351729] INFO: moduleinvoker: trade.v4 运行完成[23.907009s].

In [ ]:
# df = m1.data_2.read()#.info()
# # df = df.apply(cal_con, axis = 1)
# # df[(df['instrument']=='512010.HOF')&(df['date']>'2017-07-12')]
# # df = df[df['instrument'].isin(['512010.HOF', '159920.ZOF'])].copy()
# # [['instrument','date','w_score']].copy()
# df
# df[df['buy_condition']==1]

m2.result.best_params_

In [ ]:
m2.result.best_score_